In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import cv2, os
import numpy as np

In [2]:
import keras
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import matplotlib.pyplot as plt

Using TensorFlow backend.
/home/aakashjuseja/anaconda3/envs/first-project/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
size=256
classes=5
image_dir='train_images'
mask_dir='train_masks'
data_shape = size*size


In [4]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np
from keras.utils import Sequence
import random
percent=0.80
val_image_dir='val_images'
val_mask_dir='val_masks'
data_shape=size*size
import os

def generate_paths(image_dir,mask_dir):
    image_path = [os.path.join(image_dir, x) for x in sorted(os.listdir(image_dir)) if x.endswith('.png')]
    mask_path = [os.path.join(mask_dir, x) for x in sorted(os.listdir(mask_dir)) if x.endswith('.png')]
    image_path = np.array(image_path)
    mask_path = np.array(mask_path)
    # Break data into testing and validation
    data_size = image_path.shape[0]
    random_index = np.random.permutation(data_size)
    image_path = image_path[random_index]
    mask_path = mask_path[random_index]
    
    # first 80 percent of dataset are for training
    X_tr = image_path[:int(data_size * percent)]
    Y_tr = mask_path[:int(data_size * percent)]
    # remaining subset is for validation
    X_val = image_path[int(data_size * percent):]
    Y_val = mask_path[int(data_size * percent):]
    return X_tr,Y_tr,X_val,Y_val

def onehot_label(label):
    """
    Converts labels to onehot array.
    Returns: (h, w, classes)
    """
    classes = classes
    mask = np.eye(classes)[label]
    return mask
def hot_encoder(label):
    mask = np.zeros((size, size, classes))
    for c in range(classes):
        mask[:,:,c] = (label==c).astype(int)
        
    return mask
    
def preprocess_inputs(self, X):
    return imagenet_utils.preprocess_input(X)

def reshape_labels(y):
    return np.reshape(y, (data_shape, classes))
    

def data_gen(img_folder, mask_folder, batch_size):
  c = 0
  n = os.listdir(img_folder) #List of training images
  random.shuffle(n)
  
  while (True):
    img = np.zeros((batch_size,size , size, 3)).astype('float')
    mask = np.zeros((batch_size, size, size, 5)).astype('float')

    for i in range(c, c+batch_size): #initially from 0 to 16, c = 0. 

      train_img = cv2.imread(img_folder+'/'+n[i])/255.0
      #train_img =  cv2.resize(train_img, (size, size))# Read an image from folder and resize

      img[i-c] = train_img #add to array - img[0], img[1], and so on.
    
      #train_mask = cv2.imread(mask_folder+'/'+n[i], 0)

      train_mask = hot_encoder(cv2.resize(cv2.imread(mask_folder+'/'+n[i], 0),(size,size)))
      #train_mask = train_mask.reshape(512, 512, 1) # Add extra dimension for parity with train_img size [512 * 512 * 3]
      mask[i-c] = train_mask

    c+=batch_size
    if(c+batch_size>=len(os.listdir(img_folder))):
      c=0
      random.shuffle(n)
                  # print "randomizing again"
    yield img, mask


In [5]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Activation, Reshape, Permute
from keras.layers import Convolution2D, MaxPooling2D, UpSampling2D, Conv2D, Conv2DTranspose,concatenate,Dropout
from keras.layers import BatchNormalization


In [6]:

def conv_block(tensor, nfilters, size=3, padding='same', initializer="he_normal"):
    x = Conv2D(filters=nfilters, kernel_size=(size, size), padding=padding, kernel_initializer=initializer)(tensor)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(filters=nfilters, kernel_size=(size, size), padding=padding, kernel_initializer=initializer)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x


def deconv_block(tensor, residual, nfilters, size=3, padding='same', strides=(2, 2)):
    y = Conv2DTranspose(nfilters, kernel_size=(size, size), strides=strides, padding=padding)(tensor)
    y = concatenate([y, residual], axis=3)
    y = conv_block(y, nfilters)
    return y


def Unet(img_height, img_width, nclasses=5, filters=64):
# down
    input_layer = Input(shape=(img_height, img_width, 3), name='image_input')
    conv1 = conv_block(input_layer, nfilters=filters)
    conv1_out = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = conv_block(conv1_out, nfilters=filters*2)
    conv2_out = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = conv_block(conv2_out, nfilters=filters*4)
    conv3_out = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = conv_block(conv3_out, nfilters=filters*8)
    conv4_out = MaxPooling2D(pool_size=(2, 2))(conv4)
    conv4_out = Dropout(0.5)(conv4_out)
    conv5 = conv_block(conv4_out, nfilters=filters*16)
    conv5 = Dropout(0.5)(conv5)
# up
    deconv6 = deconv_block(conv5, residual=conv4, nfilters=filters*8)
    deconv6 = Dropout(0.5)(deconv6)
    deconv7 = deconv_block(deconv6, residual=conv3, nfilters=filters*4)
    deconv7 = Dropout(0.5)(deconv7) 
    deconv8 = deconv_block(deconv7, residual=conv2, nfilters=filters*2)
    deconv9 = deconv_block(deconv8, residual=conv1, nfilters=filters)
# output
    output_layer = Conv2D(filters=nclasses, kernel_size=(1, 1))(deconv9)
    output_layer = BatchNormalization()(output_layer)
    output_layer = Activation('softmax')(output_layer)

    model = Model(inputs=input_layer, outputs=output_layer, name='Unet')
    return model

In [7]:
kernel =3
input_shape= (size, size, 3)

In [ ]:
model=Unet(size,size, nclasses=classes,filters=64)

In [ ]:
#from keras.models import load_model
#model=load_model('unet_002.h5')

In [8]:
batch_size=10
train_gen = data_gen(image_dir,mask_dir, batch_size = batch_size)
val_gen = data_gen(val_image_dir,val_mask_dir, batch_size =batch_size)

In [9]:
model=[]
from keras.models import load_model
model=load_model('unet_001-35-0.59.h5')

In [10]:
from keras import optimizers
opt=optimizers.Adam(lr=0.00001)

In [11]:

model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=['accuracy'])
num_training_samples=np.array(os.listdir(image_dir)).shape[0]
num_epochs=500
num_validation_samples=np.array(os.listdir(val_image_dir)).shape[0]


    # load data
#(num_training_samples // batch_size)
filepath = "unet_001-{epoch:02d}-{val_loss:.2f}.h5"
tb_cb = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',save_best_only=True)

model.fit_generator(generator=train_gen,
                                      steps_per_epoch=(num_training_samples // batch_size),
                                      epochs=num_epochs,
                                      verbose=1,
                                        validation_data=val_gen,
                                        validation_steps=(num_validation_samples // batch_size),
                                      use_multiprocessing=True,
                                      workers=1,
                                      max_queue_size=1,
                                       callbacks=[tb_cb],
                                        initial_epoch=36)

Epoch 37/500
734/734 [==============================] - 703s 957ms/step - loss: 0.2514 - acc: 0.9125 - val_loss: 0.5783 - val_acc: 0.7945
Epoch 38/500
734/734 [==============================] - 684s 932ms/step - loss: 0.2423 - acc: 0.9164 - val_loss: 0.5664 - val_acc: 0.7990
Epoch 39/500
734/734 [==============================] - 686s 935ms/step - loss: 0.2359 - acc: 0.9183 - val_loss: 0.5885 - val_acc: 0.7959
Epoch 40/500
734/734 [==============================] - 696s 948ms/step - loss: 0.2327 - acc: 0.9196 - val_loss: 0.5809 - val_acc: 0.7976
Epoch 41/500
734/734 [==============================] - 698s 951ms/step - loss: 0.2269 - acc: 0.9212 - val_loss: 0.5755 - val_acc: 0.8015
Epoch 42/500
734/734 [==============================] - 701s 955ms/step - loss: 0.2251 - acc: 0.9217 - val_loss: 0.5648 - val_acc: 0.8034
Epoch 43/500
734/734 [==============================] - 703s 957ms/step - loss: 0.2228 - acc: 0.9225 - val_loss: 0.5820 - val_acc: 0.8002
Epoch 44/500
734/734 [============

734/734 [==============================] - 688s 937ms/step - loss: 0.1104 - acc: 0.9581 - val_loss: 0.7288 - val_acc: 0.7994
Epoch 164/500
734/734 [==============================] - 690s 940ms/step - loss: 0.1104 - acc: 0.9580 - val_loss: 0.7242 - val_acc: 0.8009
Epoch 165/500
734/734 [==============================] - 691s 941ms/step - loss: 0.1086 - acc: 0.9587 - val_loss: 0.7413 - val_acc: 0.7990
Epoch 166/500
734/734 [==============================] - 691s 941ms/step - loss: 0.1082 - acc: 0.9587 - val_loss: 0.7328 - val_acc: 0.8004
Epoch 167/500
734/734 [==============================] - 692s 942ms/step - loss: 0.1087 - acc: 0.9585 - val_loss: 0.7365 - val_acc: 0.8015
Epoch 168/500
734/734 [==============================] - 690s 939ms/step - loss: 0.1078 - acc: 0.9591 - val_loss: 0.7493 - val_acc: 0.8009
Epoch 169/500
734/734 [==============================] - 686s 934ms/step - loss: 0.1068 - acc: 0.9591 - val_loss: 0.7292 - val_acc: 0.8023
Epoch 170/500
734/734 [==================

KeyboardInterrupt: 